In [15]:
import pandas as pd
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import os
from collections import OrderedDict

# ------------------ Data Preparation ------------------ #
if not os.path.exists('population_cleaned.csv'):
    try:
        print("Creating cleaned data file...")
        df_raw = pd.read_csv('population.csv')
        pivot_df = df_raw.pivot(index='Country Name', columns='Year', values='Value')
        years = [year for year in range(1960, 2024) if str(year) in pivot_df.columns]
        pivot_df = pivot_df[years]
        pivot_df.to_csv('population_cleaned.csv', encoding='utf-8')
        print("File created: population_cleaned.csv")
    except FileNotFoundError:
        raise FileNotFoundError("❌ File 'population.csv' not found. Please provide the dataset!")

print("Loading data...")
df = pd.read_csv('population_cleaned.csv', index_col='Country Name')
df.columns = df.columns.astype(int)
print(f"Data loaded. Shape: {df.shape}")

# ------------------ Additional Data ------------------ #
growth_data = pd.DataFrame({
    'Country': df.index,
    'Growth_1960_2023': (df[2023] - df[1960]) / df[1960] * 100,
    'Population_2023': df[2023]
}).sort_values('Population_2023', ascending=False)

# ------------------ Dash App ------------------ #
app = dash.Dash(__name__)
app.title = "🌍 World Population Dashboard"

available_countries = df.index.tolist()
default_countries = [c for c in ['United States', 'China', 'India', 'Brazil', 'Nigeria'] if c in available_countries]

# ایجاد پالت رنگی ثابت برای کشورها
country_colors = {}
color_sequence = px.colors.qualitative.Set1 + px.colors.qualitative.Set2 + px.colors.qualitative.Set3
for i, country in enumerate(available_countries):
    country_colors[country] = color_sequence[i % len(color_sequence)]

button_style = {
    'padding': '6px 12px',
    'border': 'none',
    'borderRadius': '5px',
    'cursor': 'pointer',
    'color': 'white',
    'fontWeight': 'bold',
    'margin': '5px'
}

# ------------------ Layout ------------------ #
app.layout = html.Div([
    html.H1("🌍 World Population Dashboard", style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': '10px'}),
    html.P("Interactive visualization of global population trends (1960–2023)",
           style={'textAlign': 'center', 'color': '#7f8c8d', 'marginBottom': '30px'}),
    
    # نمایش کشورهای انتخاب شده و رنگ آنها
    html.Div(id='selected-countries-display', style={
        'padding': '10px',
        'marginBottom': '20px',
        'backgroundColor': '#f8f9fa',
        'borderRadius': '5px',
        'display': 'flex',
        'flexWrap': 'wrap',
        'justifyContent': 'center'
    }),

    html.Div([
        # Sidebar
        html.Div([
            html.H3("📊 Controls", style={'color': '#2c3e50'}),
            html.Label("Select Countries:", style={'fontWeight': 'bold'}),
            dcc.Dropdown(
                id='country-selector',
                options=[{'label': c, 'value': c} for c in available_countries],
                value=default_countries,
                multi=True,
                placeholder="Search countries..."
            ),
            html.Div([
                html.Button("Select All", id="select-all-btn", n_clicks=0,
                            style={**button_style, 'backgroundColor': '#3498db'}),
                html.Button("Clear All", id="clear-all-btn", n_clicks=0,
                            style={**button_style, 'backgroundColor': '#e74c3c'})
            ], style={'margin': '15px 0', 'display': 'flex', 'justifyContent': 'space-between'}),

            html.Hr(),
            html.H3("🎬 Animation Controls"),
            html.Div([
                html.Label("Animation Speed:", style={'fontWeight': 'bold', 'display': 'block', 'marginBottom': '5px'}),
                dcc.Slider(
                    id='animation-speed',
                    min=100, max=2000, step=100, value=800,
                    marks={100: 'Fast', 800: 'Medium', 2000: 'Slow'}
                )
            ], style={'marginBottom': '20px'}),
            
            html.Div([
                html.Label("Bubble Size Scale:", style={'fontWeight': 'bold', 'display': 'block', 'marginBottom': '5px'}),
                dcc.Slider(
                    id='bubble-size',
                    min=10, max=100, step=10, value=60,
                    marks={10: 'Small', 50: 'Medium', 100: 'Large'}
                )
            ], style={'marginBottom': '20px'}),

            html.Hr(),
            html.H3("⚙️ Options"),
            dcc.Checklist(
                id='scale-option',
                options=[{'label': 'Logarithmic Scale (Y-axis)', 'value': 'log'}],
                value=[],
                style={'fontSize': '13px'}
            ),
            
            html.Hr(),
            html.H3("📈 Data Summary"),
            html.Div(id='data-summary', style={
                'fontSize': '13px', 
                'backgroundColor': '#f8f9fa', 
                'padding': '10px', 
                'borderRadius': '5px',
                'maxHeight': '200px',
                'overflowY': 'auto'
            })
        ], style={'width': '25%', 'padding': '20px', 'backgroundColor': 'white',
                  'borderRadius': '10px', 'boxShadow': '0 2px 10px rgba(0,0,0,0.1)'}),

        # Charts
        html.Div([
            html.Div([
                html.H3("📈 Population Trends Over Time"),
                dcc.Graph(id='population-chart', style={'height': '400px'})
            ], style={'marginBottom': '30px', 'backgroundColor': 'white', 'padding': '20px',
                      'borderRadius': '10px', 'boxShadow': '0 2px 10px rgba(0,0,0,0.1)'}),
            
            html.Div([
                html.Div([
                    html.H3("📊 Population Distribution (2023)"),
                    dcc.Graph(id='pie-chart', style={'height': '300px'})
                ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                
                html.Div([
                    html.H3("🚀 Growth Since 1960 (%)"),
                    dcc.Graph(id='growth-chart', style={'height': '300px'})
                ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'marginLeft': '4%'})
            ], style={'marginBottom': '30px'}),

            html.Div([
                html.H3("🎬 Animated Population Growth (1960-2023)"),
                dcc.Graph(id='animated-chart', style={'height': '500px'})
            ], style={'backgroundColor': 'white', 'padding': '20px',
                      'borderRadius': '10px', 'boxShadow': '0 2px 10px rgba(0,0,0,0.1)'}),
        ], style={'width': '72%', 'marginLeft': '3%'})
    ], style={'display': 'flex'})
], style={'backgroundColor': '#f8f9fa', 'padding': '30px', 'minHeight': '100vh'})

# ------------------ Callbacks ------------------ #
@app.callback(
    [Output('country-selector', 'value'),
     Output('selected-countries-display', 'children'),
     Output('data-summary', 'children')],
    [Input('select-all-btn', 'n_clicks'),
     Input('clear-all-btn', 'n_clicks'),
     Input('country-selector', 'value')],
    prevent_initial_call=False
)
def update_selection(select_all, clear_all, selected_countries):
    trigger = dash.ctx.triggered_id

    if trigger == 'select-all-btn':
        selected = available_countries
    elif trigger == 'clear-all-btn':
        selected = []
    else:
        selected = selected_countries or []

    # ایجاد نمایش گرافیکی کشورهای انتخاب شده با رنگ‌هایشان
    country_display = []
    if selected:
        for country in selected:
            country_display.append(
                html.Span([
                    html.Span("■", style={'color': country_colors[country], 'fontSize': '16px', 'marginRight': '5px'}),
                    html.Span(country, style={'marginRight': '15px', 'fontSize': '12px'})
                ])
            )
    else:
        country_display = [html.Span("No countries selected", style={'color': '#7f8c8d'})]

    # خلاصه داده‌ها
    if not selected:
        summary = "Select countries to see data summary"
    else:
        summary = []
        for country in selected:
            pop_1960 = df.loc[country, 1960]
            pop_2023 = df.loc[country, 2023]
            growth = ((pop_2023 - pop_1960) / pop_1960) * 100
            summary.append(html.Div([
                html.Strong(f"{country}:"),
                html.Ul([
                    html.Li(f"1960 Population: {pop_1960:,.0f}"),
                    html.Li(f"2023 Population: {pop_2023:,.0f}"),
                    html.Li(f"Growth: {growth:.1f}%")
                ])
            ], style={'marginBottom': '10px'}))

    return selected, country_display, summary

# ------------------ Population Line Chart ------------------ #
@app.callback(
    Output('population-chart', 'figure'),
    [Input('country-selector', 'value'),
     Input('scale-option', 'value')]
)
def update_line_chart(selected_countries, scale_option):
    if not selected_countries:
        return go.Figure().update_layout(
            title="Please select at least one country",
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

    fig = go.Figure()
    for c in selected_countries:
        fig.add_trace(go.Scatter(
            x=df.columns, y=df.loc[c], name=c,
            mode='lines+markers',
            hovertemplate=f"<b>{c}</b><br>Year: %{{x}}<br>Population: %{{y:,}}<extra></extra>",
            line=dict(width=3, color=country_colors[c]),
            marker=dict(size=6, color=country_colors[c])
        ))
    
    fig.update_layout(
        xaxis_title="Year", 
        yaxis_title="Population",
        yaxis_type='log' if 'log' in scale_option else 'linear',
        plot_bgcolor='white', 
        paper_bgcolor='white',
        hovermode='x unified',
        legend=dict(
            title="Countries", 
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="right", 
            x=1,
            font=dict(size=10)
        ),
        height=400,
        xaxis=dict(
            tickmode='linear',
            dtick=10,  # نمایش هر 10 سال
            tickangle=45,  # چرخش برچسب‌ها برای جلوگیری از تداخل
            rangeslider=dict(visible=True, thickness=0.05)
        )
    )
    
    return fig

# ------------------ Small Charts ------------------ #
@app.callback(
    [Output('pie-chart', 'figure'),
     Output('growth-chart', 'figure')],
    [Input('country-selector', 'value')]
)
def update_small_charts(selected_countries):
    if not selected_countries:
        empty_fig = go.Figure().update_layout(
            plot_bgcolor='white',
            paper_bgcolor='white',
            annotations=[dict(text="Please select countries", showarrow=False, font=dict(size=16))]
        )
        return empty_fig, empty_fig
    
    pie_data = growth_data[growth_data['Country'].isin(selected_countries)]
    
    # ایجاد رنگ‌های متناسب برای نمودار پای
    pie_colors = [country_colors[country] for country in pie_data['Country']]
    
    pie_fig = px.pie(
        pie_data, 
        values='Population_2023', 
        names='Country',
        title='',
        hole=0.4,
        color='Country',
        color_discrete_map=country_colors
    )
    pie_fig.update_traces(
        textposition='inside', 
        textinfo='percent+label',
        hovertemplate="<b>%{label}</b><br>Population: %{value:,}<br>Percentage: %{percent}<extra></extra>",
        marker=dict(colors=pie_colors)
    )
    pie_fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=False,
        height=300,
        uniformtext_minsize=10,
        uniformtext_mode='hide'
    )
    
    # ایجاد نمودار رشد با رنگ‌های ثابت
    growth_fig_data = pie_data.sort_values('Growth_1960_2023', ascending=True)
    bar_colors = [country_colors[country] for country in growth_fig_data['Country']]
    
    growth_fig = px.bar(
        growth_fig_data,
        x='Growth_1960_2023',
        y='Country',
        orientation='h',
        title='',
        color='Country',
        color_discrete_map=country_colors
    )
    growth_fig.update_traces(
        hovertemplate="<b>%{y}</b><br>Growth: %{x:.1f}%<extra></extra>",
        marker_color=bar_colors
    )
    growth_fig.update_layout(
        xaxis_title="Growth Since 1960 (%)",
        yaxis_title="",
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=False,
        height=300,
        yaxis=dict(tickfont=dict(size=10))
    )
    
    return pie_fig, growth_fig

# ------------------ Animated Bubble Chart ------------------ #
@app.callback(
    Output('animated-chart', 'figure'),
    [Input('country-selector', 'value'),
     Input('animation-speed', 'value'),
     Input('bubble-size', 'value')]
)
def update_bubble_chart(selected_countries, animation_speed, bubble_size):
    if not selected_countries:
        return go.Figure().update_layout(
            title="Please select at least one country",
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

    bubble_data = df.loc[selected_countries].reset_index().melt(
        id_vars="Country Name", var_name="Year", value_name="Population")
    bubble_data["Year"] = bubble_data["Year"].astype(int)
    
    bubble_data["Growth"] = bubble_data.groupby("Country Name")["Population"].transform(
        lambda x: (x - x.iloc[0]) / x.iloc[0] * 100
    )

    fig = px.scatter(
        bubble_data,
        x="Year",
        y="Population",
        size="Population",
        color="Country Name",
        color_discrete_map=country_colors,
        hover_name="Country Name",
        animation_frame="Year",
        animation_group="Country Name",
        size_max=bubble_size,
        opacity=0.8,
        range_x=[1960, 2023],
        range_y=[bubble_data["Population"].min() * 0.9, bubble_data["Population"].max() * 1.1],
        labels={"Population": "Population", "Year": "Year"}
    )

    fig.update_layout(
        title="🌍 Animated Population Growth (1960–2023)",
        xaxis=dict(title="Year", gridcolor="lightgrey", zeroline=False, showgrid=True),
        yaxis=dict(title="Population", gridcolor="lightgrey", zeroline=False,
                   type='log' if bubble_data["Population"].max() / bubble_data["Population"].min() > 100 else 'linear'),
        plot_bgcolor='white',
        paper_bgcolor='white',
        legend=dict(
            title="Countries", 
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="right", 
            x=1,
            font=dict(size=10)
        ),
        transition={'duration': animation_speed, 'easing': 'cubic-in-out'},
        updatemenus=[dict(
            type="buttons",
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, {
                        "frame": {"duration": animation_speed, "redraw": True},
                        "fromcurrent": True,
                        "transition": {"duration": animation_speed, "easing": "cubic-in-out"}
                    }]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[[None], {
                        "frame": {"duration": 0, "redraw": False},
                        "mode": "immediate",
                        "transition": {"duration": 0}
                    }]
                )
            ]
        )],
        height=500
    )

    fig.update_traces(
        hovertemplate="<b>%{hovertext}</b><br>Year: %{x}<br>Population: %{y:,}<extra></extra>",
        marker=dict(sizemode='diameter', line=dict(width=2, color='DarkSlateGrey'))
    )

    # اضافه کردن فضای بیشتر برای جلوگیری از تداخل نوشته‌ها
    fig.update_layout(
        margin=dict(l=50, r=50, t=80, b=100),
        annotations=[dict(
            x=0.5, y=-0.2,
            xref="paper", yref="paper",
            text="Bubble size represents population size",
            showarrow=False,
            font=dict(size=10, color="grey")
        )]
    )

    return fig

# ------------------ Run ------------------ #
if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=8050)

Loading data...
Data loaded. Shape: (265, 64)
